In [2]:
import os
import numpy as np
import pandas as pd

In [ ]:
'''
1. 로그 불러옴
2. 로그의 레퍼파일이름의 유니크를 받음
3. 로그레퍼[i]에서 src file들을 입력
    3-1. 파일들은 NNNNNN.npy로 h3d 파일로 있음
    3-2. index.csv 불러와서 NNNNNN.npy를 path/to/save.npz로 변경
4. 변경된 npz를 레퍼파일의 이름폴더에 저장
5. 폴더 내에는 npz와 csv 폴더가 나뉘어져 있어야함
'''

In [112]:
#로그 파일 불러옴
log = pd.read_csv("log10.csv")
log.head()

,ref_file_name,src_file_name,Cos_Similarity,max_frame_start,max_frame_end
0,Skydive_Recovery.npy,001203.npy,0.655987,136,167
1,Skydive_Recovery.npy,001203.npy,0.654890,140,171
2,Skydive_Recovery.npy,001203.npy,0.650773,144,175
3,Skydive_Recovery.npy,008217.npy,0.650168,108,139
4,Skydive_Recovery.npy,008217.npy,0.651262,112,143


In [113]:
#레퍼 파일 유니크 값 얻음
ref_file_name_list = log.loc[:,'ref_file_name'].unique()
print(len(ref_file_name_list))
print(ref_file_name_list)




6
['Skydive_Recovery.npy' 'Ascend_Start.npy' 'Turn_LT_180.npy'
 'Look_RTUP.npy' 'Turn_RT_180.npy' 'Fly_LT.npy']


In [114]:
sim_file_list = log[(log['ref_file_name'] ==ref_file_name_list[1]) & (log['Cos_Similarity'] >= 0.73)]['src_file_name'].unique()
print(len(sim_file_list))
print(sim_file_list)


22
['006520.npy' '007482.npy' '003679.npy' '006235.npy' '004842.npy'
 '005457.npy' '000961.npy' '009483.npy' '002562.npy' '010532.npy'
 '010994.npy' '014020.npy' '005874.npy' '011940.npy' '000683.npy'
 '001930.npy' '008756.npy' '002449.npy' '011166.npy' '007867.npy'
 '000466.npy' '006084.npy']


In [115]:
idx_file = pd.read_csv('index.csv')
idx_file.head()

,source_path,start_frame,end_frame,new_name
0,./pose_data/KIT/3/kick_high_left02_poses.npy,0,117,000000.npy
1,./pose_data/humanact12/humanact12/P11G01R02F18...,0,-1,000001.npy
2,./pose_data/KIT/3/jump_left02_poses.npy,0,83,000002.npy
3,./pose_data/KIT/1747/displace_from_left_to_rig...,0,91,000003.npy
4,./pose_data/CMU/80/80_63_poses.npy,516,716,000004.npy


In [116]:
#for문 돌면서 소스패스 추가하기
source_path_list = []
for i in range(len(sim_file_list)):
    source_path = idx_file[idx_file['new_name']==sim_file_list[i]]['source_path']
    
    source_path.values[0] = source_path.values[0].replace("./pose_data", "amass_data")
    source_path.values[0] = source_path.values[0].replace(".npy", ".npz")
    # print(source_path.values[0])
    source_path_list.append(source_path.values[0])
print(source_path_list)

['amass_data/BMLmovi/Subject_65_F_MoSh/Subject_65_F_11_poses.npz', 'amass_data/BMLmovi/Subject_48_F_MoSh/Subject_48_F_11_poses.npz', 'amass_data/KIT/675/dry_right_arm_006_poses.npz', 'amass_data/BMLmovi/Subject_14_F_MoSh/Subject_14_F_16_poses.npz', 'amass_data/BMLmovi/Subject_81_F_MoSh/Subject_81_F_20_poses.npz', 'amass_data/HumanEva/S1/Static_poses.npz', 'amass_data/Eyes_Japan_Dataset/kaiwa/gesture_etc-22-mobile receive-kaiwa_poses.npz', 'amass_data/BMLmovi/Subject_70_F_MoSh/Subject_70_F_5_poses.npz', 'amass_data/Eyes_Japan_Dataset/kaiwa/pose-01-normal-kaiwa_poses.npz', 'amass_data/CMU/24/24_01_poses.npz', 'amass_data/BioMotionLab_NTroje/rub065/0031_rom_poses.npz', 'amass_data/BMLmovi/Subject_24_F_MoSh/Subject_24_F_1_poses.npz', 'amass_data/BioMotionLab_NTroje/rub066/0030_rom_poses.npz', 'amass_data/BMLmovi/Subject_6_F_MoSh/Subject_6_F_18_poses.npz', 'amass_data/BMLmovi/Subject_81_F_MoSh/Subject_81_F_4_poses.npz', 'amass_data/BMLmovi/Subject_40_F_MoSh/Subject_40_F_1_poses.npz', 'amass

In [110]:
# 저장폴더 - 최상단 폴더 설정
save_root = 'npy_cos_sim/'
# 저장폴더 - 레퍼파일 폴더명 생성
for ref_folder in ref_file_name_list:
    save_folder = os.path.join(save_root,ref_folder[:-4])
    
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)



In [120]:
import shutil
# amass 파일 폴더에서 데이터 있는지 확인하기

for source_path in source_path_list:
    
    # source_file_path = 'amass_data/CMU/114/114_16_poses.npz'
    npy_file_name = source_path.split("/")[-1]
    # print(npy_file_name)
    # 복사할 대상 경로
    destination_folder = 'npy_cos_sim/'
    # destination_file_path = os.path.join(destination_folder, str(ref_file_name_list[2][:-4]),'npz',npy_file_name)
    destination_file_path = destination_folder  + str(ref_file_name_list[1][:-4]) + '/npz/' +npy_file_name
    # print(destination_file_path)

    # 파일이 존재하는지 확인
    if os.path.exists(source_path):
        # 대상 폴더가 존재하지 않으면 생성
        # print("파일 존재합니다.")
        pass
    else:
        print("파일 안존재함")

    if not os.path.exists(destination_folder):
        print("폴더 내에 파일이 존재하지 않습니다.")
    else:
        # print(f'저장 폴더 있습니다.')
        # 파일 복사
        shutil.copy2(source_path, destination_file_path)
        print(f"파일이 {destination_file_path}에 성공적으로 복사되었습니다.")

# 있으면 그 파일 복사에서 타겟 폴더에 붙여넣기 없으면 pass하기

파일이 npy_cos_sim/Ascend_Start/npz/Subject_65_F_11_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Subject_48_F_11_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/dry_right_arm_006_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Subject_14_F_16_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Subject_81_F_20_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Static_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/gesture_etc-22-mobile receive-kaiwa_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Subject_70_F_5_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/pose-01-normal-kaiwa_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/24_01_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/0031_rom_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/Subject_24_F_1_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/0030_rom_poses.npz에 성공적으로 복사되었습니다.
파일이 npy_cos_sim/Ascend_Start/npz/

In [88]:
# 유사도 결과 npz 파일들 불러와서 csv로 바꾸는 스크립트
'''
1. npy_cos_sim 폴더를 지정
1-1. 폴더 내에는 
레퍼 이름
|--csv
|--npz
    |--파일들...
    |--파일들...
    |--파일들...
1-2. 폴더 내에 있는 
'''


In [97]:
fly_folder_list = os.listdir("npy_cos_sim")
fly_file_path = 'Ascend_Start'
# print(fly_folder_list)
sim_npz_list = os.listdir("npy_cos_sim/"+ fly_file_path+'/npz')
# print(sim_npz_list)
npz_file_list =[]
for i in range(len(sim_npz_list)):
    # file_add = os.path.join("npy_cos_sim",fly_folder_list[0],"npz",sim_npz_list[i])
    file_add = "npy_cos_sim/" + fly_folder_list[0] +"/npz/"+sim_npz_list[i]
    npz_file_list.append(file_add)

print(npz_file_list)



['npy_cos_sim/Fly_BK/npz/28_13_poses.npz', 'npy_cos_sim/Fly_BK/npz/30_23_poses.npz', 'npy_cos_sim/Fly_BK/npz/54_16_poses.npz', 'npy_cos_sim/Fly_BK/npz/54_23_poses.npz', 'npy_cos_sim/Fly_BK/npz/80_58_poses.npz', 'npy_cos_sim/Fly_BK/npz/dry_head03_poses.npz', 'npy_cos_sim/Fly_BK/npz/dry_left_arm_03_poses.npz', 'npy_cos_sim/Fly_BK/npz/dry_left_arm_05_poses.npz', 'npy_cos_sim/Fly_BK/npz/dry_left_arm_12_poses.npz', 'npy_cos_sim/Fly_BK/npz/shower_head01_poses.npz', 'npy_cos_sim/Fly_BK/npz/shower_head02_poses.npz', 'npy_cos_sim/Fly_BK/npz/shower_head03_poses.npz', 'npy_cos_sim/Fly_BK/npz/Subject_53_F_7_poses.npz', 'npy_cos_sim/Fly_BK/npz/Subject_54_F_10_poses.npz', 'npy_cos_sim/Fly_BK/npz/Subject_66_F_12_poses.npz', 'npy_cos_sim/Fly_BK/npz/Subject_6_F_18_poses.npz', 'npy_cos_sim/Fly_BK/npz/Subject_85_F_4_poses.npz', 'npy_cos_sim/Fly_BK/npz/uar1b_poses.npz', 'npy_cos_sim/Fly_BK/npz/uar3_poses.npz', 'npy_cos_sim/Fly_BK/npz/wash_right_arm_01_poses.npz']


In [98]:
# file_path = npy_cos_sim/{FLY_FILE_FOLDER}/npz/{FILENAME.npz}
def npz2csv(file_path):
    npz_data = np.load(file_path, allow_pickle=True)
    a2 = npz_data['trans']
    b2 = npz_data['poses']
    # fr = npz_data['mocap_framerate']
    result =np.concatenate((a2,b2),axis=1)

    save_file_name = file_path.replace("/npz/","/csv/")
    np.savetxt(save_file_name[:-4]+ '.csv', result, delimiter=",")  # , fmt='%s'
    print(save_file_name," 저장완료.")


In [100]:
for npzfile in npz_file_list:
    npz2csv(npzfile)

npy_cos_sim/Fly_BK/csv/28_13_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/30_23_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/54_16_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/54_23_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/80_58_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/dry_head03_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/dry_left_arm_03_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/dry_left_arm_05_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/dry_left_arm_12_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/shower_head01_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/shower_head02_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/shower_head03_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/Subject_53_F_7_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/Subject_54_F_10_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/Subject_66_F_12_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/Subject_6_F_18_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/Subject_85_F_4_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/uar1b_poses.npz  저장완료.
npy_cos_sim/Fly_BK/csv/uar3_poses.npz  저장완료.
npy_cos_sim/Fly_BK/cs